In [1]:
import math

import import_ipynb
from utils import df_from_csv, to_be_checked, is_corr_ok, plot_hist, plot_boxes, plot_lines, df_to_csv

importing Jupyter notebook from utils.ipynb


## PM2.5 Clean Data

In [2]:
# Load Raw Data
df_pm25_raw = df_from_csv("pm25_raw.csv")

# Check Data Types
print(df_pm25_raw.dtypes)

df_pm25_raw.head()

TANGARA_25CE    float64
TANGARA_260A    float64
TANGARA_4B1A    float64
TANGARA_14D6    float64
TANGARA_1EE6    float64
TANGARA_2E9A    float64
TANGARA_2FF6    float64
TANGARA_1712    float64
TANGARA_48C6    float64
TANGARA_532E    float64
TANGARA_F1AE    float64
TANGARA_06BE    float64
dtype: object


,TANGARA_25CE,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_1EE6,TANGARA_2E9A,TANGARA_2FF6,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
DATETIME,,,,,,,,,,,,
2023-09-26 00:00:30-05:00,1.0,6.0,5.0,7.0,2.0,11.0,7.0,14.0,3.0,7.0,5.0,0.0
2023-09-26 00:01:00-05:00,1.0,6.0,7.0,6.0,3.0,12.0,8.0,13.0,7.0,9.0,5.0,0.0
2023-09-26 00:01:30-05:00,1.0,7.0,8.0,6.0,4.0,11.0,8.0,12.0,7.0,NaN,5.0,0.0
2023-09-26 00:02:00-05:00,0.0,7.0,8.0,6.0,4.0,11.0,8.0,14.0,7.0,11.0,5.0,0.0
2023-09-26 00:02:30-05:00,1.0,6.0,8.0,6.0,3.0,9.0,9.0,13.0,5.0,10.0,5.0,0.0


## Descriptive Statistics

In [3]:
# Describe Data
df_pm25_raw.describe()

,TANGARA_25CE,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_1EE6,TANGARA_2E9A,TANGARA_2FF6,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,8.000000,9.000000,9.0
mean,0.888889,6.333333,7.333333,5.777778,3.000000,10.666667,8.777778,13.222222,5.777778,9.375000,4.888889,0.0
std,0.333333,0.500000,1.000000,0.666667,0.707107,1.000000,1.092906,0.666667,1.394433,1.187735,0.333333,0.0
min,0.000000,6.000000,5.000000,5.000000,2.000000,9.000000,7.000000,12.000000,3.000000,7.000000,4.000000,0.0
25%,1.000000,6.000000,7.000000,5.000000,3.000000,10.000000,8.000000,13.000000,5.000000,9.000000,5.000000,0.0
50%,1.000000,6.000000,8.000000,6.000000,3.000000,11.000000,9.000000,13.000000,6.000000,9.500000,5.000000,0.0
75%,1.000000,7.000000,8.000000,6.000000,3.000000,11.000000,10.000000,14.000000,7.000000,10.000000,5.000000,0.0
max,1.000000,7.000000,8.000000,7.000000,4.000000,12.000000,10.000000,14.000000,7.000000,11.000000,5.000000,0.0
